***GENERATED CODE FOR creditcardfrauddetection PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run creditcardfrauddetectionHooks.ipynb
try:
	#sourcePreExecutionHook()

	creditcardtrainingdata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1689084397531_creditcardTrainingData.csv', 'filename': 'creditcardTrainingData.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Finance Models/Credit Card Fraud Detection - Classification/creditcardTrainingData.csv', 'viewFileName': 'creditcardTrainingData.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run creditcardfrauddetectionHooks.ipynb
try:
	#transformationPreExecutionHook()

	creditcardfrauddetectionautofe = TransformationMain.run(creditcardtrainingdata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Time", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "914.64", "stddev": "119.36", "min": "711", "max": "1112", "missing": "0"}, "updatedLabel": "Time"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.27", "stddev": "1.32", "min": "-6.169664484", "max": "1.553492342", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.21", "stddev": "1.05", "min": "-5.137873751", "max": "6.118940075", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V3", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.88", "stddev": "0.95", "min": "-2.824895145", "max": "4.017561418", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V4", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.16", "stddev": "1.23", "min": "-3.421194859", "max": "3.551137319", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V5", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.07", "stddev": "1.08", "min": "-9.74943959", "max": "4.615393137", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V6", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.01", "stddev": "1.13", "min": "-2.929059063", "max": "6.456410462", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V7", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.18", "stddev": "0.94", "min": "-4.417815433", "max": "10.78408765", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V8", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.08", "stddev": "0.92", "min": "-12.25815843", "max": "1.749298354", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V8"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V9", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.01", "stddev": "0.94", "min": "-3.110515096", "max": "6.450991507", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V9"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V10", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.03", "stddev": "1.08", "min": "-3.563577697", "max": "11.90686815", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V10"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V11", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.2", "stddev": "0.95", "min": "-1.938139162", "max": "2.768579466", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V11"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V12", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.32", "stddev": "0.62", "min": "-1.981435107", "max": "1.901406633", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V12"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V13", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.11", "stddev": "0.9", "min": "-2.847787057", "max": "3.182540505", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V13"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V14", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.12", "stddev": "0.75", "min": "-6.33106882", "max": "1.383447637", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V14"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V15", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.16", "stddev": "0.89", "min": "-2.979673551", "max": "2.694381552", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V15"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V16", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.08", "stddev": "0.75", "min": "-3.576360878", "max": "2.165175204", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V16"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V17", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.14", "stddev": "0.63", "min": "-2.203125493", "max": "2.607974807", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V17"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V18", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.15", "stddev": "0.73", "min": "-3.510029069", "max": "2.41756394", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V18"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V19", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.07", "stddev": "0.8", "min": "-2.801716325", "max": "3.097748598", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V19"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V20", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.1", "stddev": "0.51", "min": "-2.203675593", "max": "4.843083043", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V20"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V21", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.06", "stddev": "0.5", "min": "-3.509804038", "max": "4.406196272", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V21"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V22", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.13", "stddev": "0.59", "min": "-1.866847456", "max": "1.957759397", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V22"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V23", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.06", "stddev": "0.38", "min": "-4.020300069", "max": "4.095021042", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V23"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V24", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.01", "stddev": "0.6", "min": "-1.831952587", "max": "1.083155808", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V24"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V25", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.12", "stddev": "0.38", "min": "-1.577383889", "max": "1.069105157", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V25"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V26", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.05", "stddev": "0.42", "min": "-1.096647654", "max": "1.556236445", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V26"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V27", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.02", "stddev": "0.35", "min": "-2.887660656", "max": "2.464962198", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V27"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V28", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.01", "stddev": "0.24", "min": "-1.6836546", "max": "1.304311877", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V28"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "75.53", "stddev": "186.11", "min": "0.0", "max": "2452.03", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Class", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.1", "stddev": "0.3", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Class"}]}))

	#transformationPostExecutionHook(creditcardfrauddetectionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run creditcardfrauddetectionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(creditcardfrauddetectionautofe, ["Time", "V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "V10", "V11", "V12", "V13", "V14", "V15", "V16", "V17", "V18", "V19", "V20", "V21", "V22", "V23", "V24", "V25", "V26", "V27", "V28", "Amount"], "Class")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

